# Random Forest

This notebook is to help you setup your first model on the bit.bio colony segmentation features.
The model will be a random forest; it is a easy model to start with for data without a clear or uniform scale between the parameters.

First of all the loading of the data.
We load the enriched data and drop ImageNumber, Metadata_Well, and ObjectNumber because these are not things we want our model to react on.
Metadata_TimePoint is also removed for now because it is not a number.

Next up all location data is removed.
If we do not do that the model could learn that a colony is the right top is good, and the left bottom is bad.

In [2]:
import os
import sys
from pathlib import Path
basefolder_loc = Path(os.path.abspath('')).parents[0]
sys.path.append(str(basefolder_loc))

from feature_engineering import load_enriched_data, remove_all_location_info
bitbio_data = load_enriched_data()

# Remove some columens that are not numberic indicators
bitbio_data = bitbio_data.drop(
    columns=[
            "ImageNumber",  
            "Metadata_Well",
            "ObjectNumber", 
            "Metadata_TimePoint"
        ]
)

bitbio_data = remove_all_location_info(bitbio_data)

bitbio_data.head()

,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_EulerNumber,AreaShape_Extent,AreaShape_FormFactor,AreaShape_MajorAxisLength,AreaShape_MaxFeretDiameter,...,Texture_Variance_Phase_3_02_256,Texture_Variance_Phase_3_03_256,Texture_Variance_Phase_5_00_256,Texture_Variance_Phase_5_01_256,Texture_Variance_Phase_5_02_256,Texture_Variance_Phase_5_03_256,is_good,SampleName,ColonyName,Circularity
Good_1_00d12h00m.tif,1423,7597,13.697258,0.958639,42.565477,1,0.187311,0.073007,120.073586,116.361506,...,49.908811,52.515037,55.191058,58.169413,52.687000,51.290731,True,Good_1,Good_1_1,0.073007
Good_1_01d00h00m.tif,1288,2624,8.649761,0.997509,40.496080,1,0.490854,0.115610,180.713061,163.370744,...,12.695967,12.009367,15.489024,15.462723,12.422264,15.426300,True,Good_1,Good_1_1,0.115610
Good_1_01d04h00m.tif,2916,12420,7.879338,0.995852,60.932475,1,0.234783,0.126914,213.572235,211.009479,...,22.468556,20.122497,21.288325,22.808554,20.986270,20.334934,True,Good_1,Good_1_1,0.126914
Good_1_02d04h00m.tif,1398,3312,2.397711,0.945498,42.189914,1,0.422101,0.417064,82.411836,76.967526,...,110.477651,113.082678,110.650754,113.107319,113.482992,117.162961,True,Good_1,Good_1_1,0.417064
Good_1_02d08h00m.tif,1683,4180,2.564627,0.944304,46.291059,1,0.402632,0.389920,89.759893,83.186537,...,69.727323,68.747062,68.624614,67.173322,70.372928,68.000640,True,Good_1,Good_1_1,0.389920


In [3]:
bitbio_data.groupby("SampleName").count()

,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_EulerNumber,AreaShape_Extent,AreaShape_FormFactor,AreaShape_MajorAxisLength,AreaShape_MaxFeretDiameter,...,Texture_Variance_Phase_3_01_256,Texture_Variance_Phase_3_02_256,Texture_Variance_Phase_3_03_256,Texture_Variance_Phase_5_00_256,Texture_Variance_Phase_5_01_256,Texture_Variance_Phase_5_02_256,Texture_Variance_Phase_5_03_256,is_good,ColonyName,Circularity
SampleName,,,,,,,,,,,,,,,,,,,,,
Bad_1,411,411,411,411,411,411,411,411,411,411,...,411,411,411,411,411,411,411,411,411,411
Bad_2,148,148,148,148,148,148,148,148,148,148,...,148,148,148,148,148,148,148,148,148,148
Good_1,99,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99
Good_10,142,142,142,142,142,142,142,142,142,142,...,142,142,142,142,142,142,142,142,142,142
Good_11,117,117,117,117,117,117,117,117,117,117,...,117,117,117,117,117,117,117,117,117,117
Good_12,151,151,151,151,151,151,151,151,151,151,...,151,151,151,151,151,151,151,151,151,151
Good_13,101,101,101,101,101,101,101,101,101,101,...,101,101,101,101,101,101,101,101,101,101
Good_14,174,174,174,174,174,174,174,174,174,174,...,174,174,174,174,174,174,174,174,174,174
Good_15,182,182,182,182,182,182,182,182,182,182,...,182,182,182,182,182,182,182,182,182,182


In [4]:
bitbio_data.groupby("ColonyName").count()

,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_EulerNumber,AreaShape_Extent,AreaShape_FormFactor,AreaShape_MajorAxisLength,AreaShape_MaxFeretDiameter,...,Texture_Variance_Phase_3_01_256,Texture_Variance_Phase_3_02_256,Texture_Variance_Phase_3_03_256,Texture_Variance_Phase_5_00_256,Texture_Variance_Phase_5_01_256,Texture_Variance_Phase_5_02_256,Texture_Variance_Phase_5_03_256,is_good,SampleName,Circularity
ColonyName,,,,,,,,,,,,,,,,,,,,,
Bad_1_1,86,86,86,86,86,86,86,86,86,86,...,86,86,86,86,86,86,86,86,86,86
Bad_1_10,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
Bad_1_11,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Bad_1_12,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Bad_1_13,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Good_9_5,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
Good_9_6,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
Good_9_7,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [5]:
import random

all_colony_names = bitbio_data.ColonyName.unique()
all_good_colony_names = list(filter(lambda x: x.startswith("Good_"), all_colony_names))
all_bad_colony_names = list(filter(lambda x: x.startswith("Bad_"), all_colony_names))

random.shuffle(all_good_colony_names)
random.shuffle(all_bad_colony_names)

training_colony_names = all_good_colony_names[:-10] + all_bad_colony_names[:-10]
test_colony_names = all_good_colony_names[-10:] + all_bad_colony_names[-10:]

training_data = bitbio_data[bitbio_data['ColonyName'].isin(training_colony_names)]
test_data = bitbio_data[bitbio_data['ColonyName'].isin(test_colony_names)]

training_data = training_data.drop(columns=[
        "ColonyName",
        "SampleName"
    ])
test_data = test_data.drop(columns=[
        "ColonyName",
        "SampleName"
    ])

In [6]:
from sklearn.ensemble import RandomForestClassifier

y = training_data.is_good
X = training_data.drop(
    columns=[
        "is_good",
    ]
)
clf = RandomForestClassifier(max_depth=12, random_state=0)
clf.fit(X, y)

RandomForestClassifier(max_depth=12, random_state=0)

In [7]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

p = clf.predict(X)
print("Precision", precision_score(test_data.is_good, p))
print("Recall", recall_score(test_data.is_good, p))
confusion_matrix(y, p)

array([[ 281,   51],
       [   0, 3251]], dtype=int64)

In [8]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
p = clf.predict(test_data.drop(columns=["is_good"]))
print("Precision", precision_score(test_data.is_good, p))
print("Recall", recall_score(test_data.is_good, p))
confusion_matrix(test_data.is_good, p)

Precision 0.5656934306569343
Recall 1.0


array([[108, 119],
       [  0, 155]], dtype=int64)